In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
import math

In [30]:
prog = pd.read_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\proglist.csv')
ped = pd.read_csv("/Users/Jeanne/Documents/personal/Auction Results/pedigree.csv")

In [45]:
prog = prog.drop('Unnamed: 0',1)
prog.shape

(18813, 2)

In [46]:
recall = prog
keep = prog.drop_duplicates(keep='first')
keep.shape

(15667, 2)

In [51]:
ped.shape

(16745, 16)

In [52]:
ped['Name'] = ped['Name'].str.replace('_',' ')
ped = ped.drop('Unnamed: 0',1)
ped.head()

,Name,S,SS,SSS,SSD,SD,SDS,SDD,D,DS,DSS,DSD,DD,DDS,DDD
0,Nannerl,Valid Appeal,In Reality,Intentionally,My Dear Girl,Desert Trial,Moslem Chief,Scotch Verdict,Alloutette,Proud Birdie,Proud Clarion,Bernie Bird,Madame Defage,Bayou Bourg,Lets Misbehave
1,Note to Mimi,Notebook,Well Decorated,Raja Baba,Paris Breeze,Mobcap,Tom Rolfe,Cap and Bells,Mimi La Sardine,Tanks Prospect,Mr. Prospector,Midnight Pumpkin,Fran Nasra,Francis S.,Queen Nasra
2,Parkdale,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found
3,Party Ribbon,Java Gold,Key to the Mint,Graustark,Key Bridge,Javamine,Nijinsky II,Dusky Evening,Tappity Tap,Mr. Prospector,Raise a Native,Gold Digger,Colony Club,Tom Rolfe,Amerigo Lady
4,Pearlwood,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found,no data found


In [40]:
len(prog['Name'].unique())
len(ped['Name'].unique())

16745

In [43]:
dupes = prog[prog.duplicated(['Name'])]
dupes.sort_values('Name')

,Unnamed: 0,Name,Progeny_raw
6537,19946,A Day in Oz,NaN
11571,24980,A J's Gal,NaN
9287,22696,A J's Girl,NaN
17301,30710,A J's Honey,NaN
13466,26875,A Little Smart,NaN
...,...,...,...
13460,26869,Zinned,NaN
15466,28875,Zipp Six Fifty,NaN
8166,21575,Zizi Top (GB),NaN
14391,27800,Zoila,NaN


In [12]:
prog[prog['Progeny_raw'] == np.nan]

,Unnamed: 0,Name,Progeny_raw


In [90]:
te = prog['Progeny_raw'].tolist()
st = [str(x) for x in te]
new = [x for x in st if x != 'nan']
new

['Burmilla (USA)\nLexington Girl (USA)\nMagical Masquerade (USA)\nMagicalmysterycat (USA)\nStep Fast (USA)',
 'Case Notes (USA)\nHello Hannah (USA)\nMotion Play (USA)\nSiren Cove (USA)\n ',
 'Tiny Bear (USA)\nCajun Infinity (USA)\n     ',
 'Bandi Kay (USA)\nHandsome Yansom (USA)\nWhite Scarf (USA)\n   ',
 'Morganstern (USA)\n       ',
 "Chief Lion (USA)\nRamona's Wildcat (USA)\nKairos (USA)\nMercadian (USA)\nTrueno Jak (USA)",
 'Absoulute Heaven (USA)\nPaulette Caveat (USA)\nPrince Dubai (USA)\nPrincess Maura (USA)\n ',
 'Privacy (USA)\nPrivate Lake (USA)\n     ',
 'Ribo Bobo (USA)\nBrother Bobo (USA)\nSaponetta Bobo (USA)\nStrategic Bobo (USA)\n ',
 'False Dilemma (USA)\nMedford (USA)\n     ',
 'No data to display',
 "Drift Away (USA)\nLaffin' Place (USA)\nSeven Ten Split (USA)\n   ",
 'Prince Raja (USA)\n       ',
 "Daddy's Honor (USA)\nGlacken Road (USA)\nHer Royal Honor (USA)\n   ",
 'Sr. Quisqueyano (USA)\nLittle Mandate (USA)\n     ',
 "Brooke's Halo (USA)\nHenry The Great (USA)\

In [91]:
g = list()
for i in new:
    parser = i.split('\n')
    for p in parser:
        g.append(p.replace(r' (USA)',''))

g = [x for x in g if not 'Page 1 of' in x]
g

['Burmilla',
 'Lexington Girl',
 'Magical Masquerade',
 'Magicalmysterycat',
 'Step Fast',
 'Case Notes',
 'Hello Hannah',
 'Motion Play',
 'Siren Cove',
 ' ',
 'Tiny Bear',
 'Cajun Infinity',
 '     ',
 'Bandi Kay',
 'Handsome Yansom',
 'White Scarf',
 '   ',
 'Morganstern',
 '       ',
 'Chief Lion',
 "Ramona's Wildcat",
 'Kairos',
 'Mercadian',
 'Trueno Jak',
 'Absoulute Heaven',
 'Paulette Caveat',
 'Prince Dubai',
 'Princess Maura',
 ' ',
 'Privacy',
 'Private Lake',
 '     ',
 'Ribo Bobo',
 'Brother Bobo',
 'Saponetta Bobo',
 'Strategic Bobo',
 ' ',
 'False Dilemma',
 'Medford',
 '     ',
 'No data to display',
 'Drift Away',
 "Laffin' Place",
 'Seven Ten Split',
 '   ',
 'Prince Raja',
 '       ',
 "Daddy's Honor",
 'Glacken Road',
 'Her Royal Honor',
 '   ',
 'Sr. Quisqueyano',
 'Little Mandate',
 '     ',
 "Brooke's Halo",
 'Henry The Great',
 'Milliondollarbill',
 'Von Carldawn',
 ' ',
 'Ally',
 'Malitzin',
 '     ',
 'Brewmistress',
 'Emile',
 'Flurtini',
 'Sharp Trip',
 ' '

In [92]:
len(g)

40185

In [103]:
for i in range(0,len(g)):
    g[i] = g[i].replace(' ', '_')
    
headers = ['Date', 'Horse', 'Fin', 'Tr', 'Dist.', 'Sf.', 'Race', 'Cnd', '1st', '2nd', '3rd', 'Time']
df = pd.DataFrame(columns=headers) # creates master dataframe 

prog428 = pd.read_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\428prog.csv')
remove = list(prog428['Horse'])
for r in range(0,len(remove)):
    remove[r] = remove[r].replace(' ', '_')
    
new = [x for x in g if x not in remove]

len(new)

18095

In [ ]:
#df = pd.DataFrame(columns=headers) # creates master dataframe 

from selenium.webdriver.chrome.options import Options

path_to_extension = r'C:\Users\Jeanne\Documents\personal\Auction Results\3.10.2_0'

chrome_options = Options()
chrome_options.add_argument('load-extension=' + path_to_extension)

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.create_options()

#driver = webdriver.Chrome('/Users/Jeanne/Downloads/chromedriver')

for i in new:
    hname = i
    url = 'https://www.horseracingnation.com/horse/' + hname
    driver.get(url)

    z = driver.find_elements_by_xpath('//td[@id="1-1"]')
    z2 = driver.find_elements_by_xpath('//td[@class="dxgvDataRow "]')
    if (len(z) == 1 and len(z2) > 0):
        # locate tr
        tablerow = driver.find_elements_by_xpath('//tr[@class="dxgvDataRow "]')
        for row in tablerow:
        # tdata = [td.text for td in row.find_elements_by_class_name('dxgv')]
          tdata = [td.text for td in row.find_elements_by_xpath('//td[@class="dxgv"]')]

        if len(tdata) == 125 | len(tdata) < 12:
            r = list()
            r.append(np.nan)
            r.append(hname.replace('_',' '))
            for a in range(10):
                r.append(np.nan)
            to_append = r
            df_length = len(df)
            df.loc[df_length] = to_append
            
        else:
            x = int(len(tdata) / 12)
            temp = pd.DataFrame(np.array(tdata).reshape(x,12), columns = headers)
            df = df.append(temp) # appends to master dataframe
        
    else:
        r = list()
        r.append(np.nan)
        r.append(hname.replace('_',' '))
        for a in range(10):
            r.append(np.nan)
        to_append = r
        df_length = len(df)
        df.loc[df_length] = to_append
        
    recall = df    
    
driver.close()
    
df.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\428prog.csv')


<ipython-input-104-619573a04ed2>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)


In [99]:
df.shape

(14187, 12)

In [100]:
df.to_csv(r'C:\Users\Jeanne\Documents\personal\Auction Results\428prog.csv')